**Loading the Model**

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 15.9 MB/s eta 0:00:00


In [13]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
from torch.utils.data import DataLoader, Dataset

In [14]:
#Loading the Domain Specific Trained Model 
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
# model.load_state_dict(torch.load('Path to stored weights - Domain Training'))
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


**Dataset Preparation**

In [4]:
#Loading the datasets
!unzip '/content/drive/MyDrive/591 Project /cosmetics.csv.zip'
!unzip '/content/drive/MyDrive/591 Project /sephora_website_dataset.csv.zip'

Archive:  /content/drive/MyDrive/591 Project /cosmetics.csv.zip
  inflating: cosmetics.csv           
Archive:  /content/drive/MyDrive/591 Project /sephora_website_dataset.csv.zip
  inflating: sephora_website_dataset.csv  


In [15]:
import pandas as pd
df1 = pd.read_csv('/content/cosmetics.csv')
df2 = pd.read_csv('/content/sephora_website_dataset.csv')

In [ ]:
df1.head()

In [7]:
df2.head()

,id,brand,category,name,size,rating,number_of_reviews,love,price,value_price,...,MarketingFlags,MarketingFlags_content,options,details,how_to_use,ingredients,online_only,exclusive,limited_edition,limited_time_offer
0,2218774,Acqua Di Parma,Fragrance,Blu Mediterraneo MINIATURE Set,5 x 0.16oz/5mL,4.0,4,3002,66.0,75.0,...,True,online only,no options,This enchanting set comes in a specially handc...,Suggested Usage:-Fragrance is intensified by t...,Arancia di Capri Eau de Toilette: Alcohol Dena...,1,0,0,0
1,2044816,Acqua Di Parma,Cologne,Colonia,0.7 oz/ 20 mL,4.5,76,2700,66.0,66.0,...,True,online only,- 0.7 oz/ 20 mL Spray - 1.7 oz/ 50 mL Eau d...,An elegant timeless scent filled with a fresh-...,no instructions,unknown,1,0,0,0
2,1417567,Acqua Di Parma,Perfume,Arancia di Capri,5 oz/ 148 mL,4.5,26,2600,180.0,180.0,...,True,online only,- 1oz/30mL Eau de Toilette - 2.5 oz/ 74 mL E...,Fragrance Family: Fresh Scent Type: Fresh Citr...,no instructions,Alcohol Denat.- Water- Fragrance- Limonene- Li...,1,0,0,0
3,1417617,Acqua Di Parma,Perfume,Mirto di Panarea,2.5 oz/ 74 mL,4.5,23,2900,120.0,120.0,...,True,online only,- 1 oz/ 30 mL Eau de Toilette Spray - 2.5 oz/...,Panarea near Sicily is an an island suspended ...,no instructions,unknown,1,0,0,0
4,2218766,Acqua Di Parma,Fragrance,Colonia Miniature Set,5 x 0.16oz/5mL,3.5,2,943,72.0,80.0,...,True,online only,no options,The Colonia Miniature Set comes in an iconic A...,Suggested Usage:-Fragrance is intensified by t...,Colonia: Alcohol Denat.- Water- Fragrance- Lim...,1,0,0,0


In [16]:
df2.rename(columns = {'ingredients':'Ingredients', 'category':'Label'}, inplace = True)

In [17]:
df2 = df2[['Ingredients', 'Label']]
df1 = df1[['Ingredients', 'Label']]

In [18]:
dataset = pd.concat([df1, df2])

In [11]:
dataset.head()

,Ingredients,Label
0,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",Moisturizer
1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",Moisturizer
2,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",Moisturizer
3,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",Moisturizer
4,"Water, Snail Secretion Filtrate, Phenyl Trimet...",Moisturizer


In [12]:
len(dataset)

10640

In [29]:
dataset['Label'].unique()

array(['Moisturizer', 'Cleanser', 'Treatment', 'Face Mask', 'Eye cream',
       'Sun protect', 'Fragrance', 'Cologne', 'Perfume',
       'Body Mist & Hair Mist', 'Body Lotions & Body Oils',
       'Body Sprays & Deodorant', 'Perfume Gift Sets', 'no category',
       'Rollerballs & Travel Size', 'Lip Balm & Treatment',
       'Lotions & Oils', 'Eye Palettes', 'Highlighter', 'Cheek Palettes',
       'Lipstick', 'Face Serums', 'Moisturizers', 'Value & Gift Sets',
       'Eye Creams & Treatments', 'Face Sunscreen',
       'Lip Balms & Treatments', 'Mini Size', 'Face Masks',
       'Face Wash & Cleansers', 'Decollete & Neck Creams', 'Face Oils',
       'Hand Cream & Foot Cream', 'Face Primer', 'Color Correct',
       'Mists & Essences', 'Tinted Moisturizer', 'Concealer',
       'Beauty Supplements', 'Facial Peels', 'Exfoliators', 'Conditioner',
       'Shampoo', 'Hair Styling Products', 'Scalp & Hair Treatments',
       'Hair Masks', 'Hair Spray', 'Hair Oil', 'Hair Primers',
       'Dry Sha

In [ ]:
#Have to preprocess/combine some labels

In [28]:
len(dataset['Label'].unique())

149

In [ ]:
# import tensorflow as tf
# import pandas as pd
# from transformers import BertTokenizer

# # Load the data
# data = pd.read_csv('data.csv')

# # Initialize the BERT tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# # Tokenize the text data
# tokenized_data = data['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

# # Pad the tokenized sequences
# padded_data = tf.keras.preprocessing.sequence.pad_sequences(tokenized_data, maxlen=128, padding='post')

# # Convert the class labels to numerical values
# labels = pd.get_dummies(data['label']).values

# # Split the data into training, validation, and testing sets
# train_dataset = padded_data[:8000]
# train_labels = labels[:8000]
# val_dataset = padded_data[8000:9000]
# val_labels = labels[8000:9000]
# test_dataset = padded_data[9000:]
# test_labels = labels[9000:]

# # Convert the data to the format that BERT accepts
# train_input = tf.data.Dataset.from_tensor_slices((train_dataset, train_labels))
# train_input = train_input.shuffle(10000).batch(32).repeat(2)

# val_input = tf.data.Dataset.from_tensor_slices((val_dataset, val_labels))
# val_input = val_input.batch(32)

# test_input = tf.data.Dataset.from_tensor_slices((test_dataset, test_labels))
# test_input = test_input.batch(32)

# # Save the preprocessed dataset
# tf.data.experimental.save(train_input, 'train_data')
# tf.data.experimental.save(val_input, 'val_data')
# tf.data.experimental.save(test_input, 'test_data')


In [ ]:
# Preprocessing 
# 1) Converting labels into numberical values
# 2) Data Split
# 3) Tokenizing - includes padding and truncation

labels = pd.get_dummies(dataset['Label']).values

train_data = dataset[:8000]
train_labels= labels[:8000]

val_data= dataset[8000:9000]
val_labels= labels[8000:9000]

test_data= dataset[9000:]
test_labels= labels[9000:]

class Ingredients(Dataset):
  def __init__(self, data, labels, tokenizer):
    self.data = data
    self.labels = labels
    self.tokenizer = tokenizer
  def __getitem__(self, index):
    text = dataset.iloc[index, 'Ingredients']
    input = self.tokenizer(text, return_tensors='pt', add_special_tokens=True, max_length=512, padding='max_length', truncation=True)
    input_ids = input.input_ids.squeeze()
    attention_mask = input.attention_mask.squeeze()
    label = self.labels[index]
    return {'input_ids' : input_ids, 'attention_mask': attention_mask, 'labels' : label}
  def __len__(self):
        return len(self.data)

train_dataset = Ingredients(train_data, train_labels)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

val_dataset = Ingredients(val_data, val_labels)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

test_dataset = Ingredients(test_data, test_labels)
train_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

Data Cleaning: Remove any unwanted characters, extra white spaces, or non-UTF-8 encoded text.

Data Tokenization: Tokenize the text data into subwords using the BERT tokenizer.

Data Padding: Pad the tokenized sequences to a fixed length, ensuring that all input sequences have the same length. This is necessary because BERT takes fixed-length inputs.

Data Labeling: Convert the class labels into numerical values.

Data Partitioning: Split the data into training, validation, and testing sets.

Data Conversion: Convert the data into the format that BERT accepts. BERT requires inputs in the form of token ids, segment ids, and attention masks.

Token ids represent the numerical ids of the tokens obtained from the tokenizer. Segment ids differentiate between the two input sequences in a sequence-pair classification task. Attention masks indicate which tokens should be attended to and which should be ignored.

Save the preprocessed dataset: Save the preprocessed dataset in a format that can be easily loaded for model training and evaluation.

In [ ]:
#Perform Data Encoding 

In [ ]:
# train_dataset = torch.utils.data.TensorDataset(
#     torch.tensor(train_encodings['input_ids']),
#     torch.tensor(train_encodings['attention_mask']),
#     torch.tensor(train_labels))

In [ ]:
#Do the same for validation dataset 

**Dataset Split**

**Model Architecture Changes**

In [20]:
import torch.nn as nn

In [21]:
class BertClassifier(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BertClassifier, self).__init__()
        self.bert = bert_model
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        logits = self.classifier(pooled_output)
        return logits

In [30]:
model = BertClassifier(model,149)

In [31]:
print(model)

BertClassifier(
  (bert): BertForMaskedLM(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
           

**Model Training**

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= 'add train dataset',
    eval_dataset= 'add test dataset',
    tokenizer= tokenizer)

trainer.train()

**Referred Code Documentaion**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertForMaskedLM

# Set the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Replace the classifier layer with a new one for classification
model.classifier = nn.Sequential(
    nn.Linear(768, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, 2)
)

# Move the model to the device
model.to(device)

# Set up the optimizer
optimizer = optim.Adam(model.parameters(), lr=2e-5)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Load the training data
train_data = ...

# Train the model
model.train()
for epoch in range(num_epochs):
    for inputs, labels in train_data:
        # Move the inputs and labels to the device
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)[0]
        
        # Compute the loss
        loss = criterion(outputs.view(-1, 2), labels.view(-1))
        
        # Backward pass and update the parameters
        loss.backward()
        optimizer.step()

# Evaluate the model on the test data
test_data = ...
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_data:
        # Move the inputs and labels to the device
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(inputs)[0]
        
        # Predict the labels
        _, predicted = torch.max(outputs.data, 1)
        
        # Compute the accuracy
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Test accuracy: {accuracy:.2f}%')


Fine-tune the pre-trained BertForMaskedLM model on your custom dataset. This involves training the model to classify the input data into the desired classes. You can use a technique called transfer learning to accomplish this. During transfer learning, you use the pre-trained model's weights as a starting point, and then train the model on your custom dataset. This helps the model learn the patterns specific to your dataset and improve its accuracy.

Prepare the data for classification. This involves converting the raw data into a format that can be fed into the BertForMaskedLM model. You can tokenize the text data using the same tokenizer that was used during pre-training, and then convert it to numerical format.

Pass the preprocessed data through the fine-tuned BertForMaskedLM model to get predictions for each input. The output of the model can be a probability distribution over the classes or a single predicted class.

Evaluate the performance of the model on the custom dataset. You can use metrics such as accuracy, precision, recall, and F1 score to evaluate the model's performance.

Iterate and improve the model as necessary. You may need to adjust the hyperparameters or architecture of the model to improve its performance on the custom dataset.

Overall, fine-tuning a pre-trained BertForMaskedLM model can be an effective approach for classification tasks on custom datasets, particularly when there is limited labeled data available.
